# 📓 ACCESS-ESM15-write-zarr

**Author:** Thomas Moore  
**Date:** 2025-10-31
**Updated:** YYYY-MM-DD (if applicable)  
**Environment:** `pangeo_csepta` running on `Gadi` ARE  
**Tags:** sandbox, ARD, ACCESS-ESM1.5

---

### 📘 Description

This notebook attempts to bring together all the disperate workplows for this task over the year(s).  Issue: [https://github.com/Thomas-Moore-Creative/Pacific-Tuna-Climate-Response/issues/51](https://github.com/Thomas-Moore-Creative/Pacific-Tuna-Climate-Response/issues/51) and sub-issues

In [1]:
Author = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

# Software

### ACDtools

In [2]:
!pip install --user -e /g/data/es60/users/thomas_moore/code/ACDtools

Obtaining file:///g/data/es60/users/thomas_moore/code/ACDtools
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ACDtools (pyproject.toml) ... done
  Created wheel for ACDtools: filename=acdtools-0.1-0.editable-py3-none-any.whl size=3553 sha256=1805a3d52f2b8a862139d103c2a41599924f56f2d86177239aacf8f3d895b8cf
  Stored in directory: /scratch/es60/thomas_moore/tmp/pip-ephem-wheel-cache-maay9erg/wheels/b6/a3/f2/6ce45fbdc116ad50e421d6a11cb060cc796e867501807af446
Successfully built ACDtools
  Attempting uninstall: ACDtools
    Found existing installation: ACDtools 0.1
    Uninstalling ACDtools-0.1:
      Successfully uninstalled ACDtools-0.1


In [3]:
# Enable autoreload in the notebook
%load_ext autoreload
%autoreload 1 
%aimport ACDtools.util
%aimport ACDtools.ard
%aimport ACDtools.ocean
%aimport ACDtools.plot
# Importing from your local package util.py
from ACDtools import util
from ACDtools import ard
from ACDtools import ocean
from ACDtools import plot

In [10]:
import datetime
from pprint import pprint
import intake

## Cluster

In [5]:
client, cluster = util.start_dask_cluster_from_config('netcdf_work')

Cluster started with 14 workers.
Dashboard available at: /proxy/8787/status


## Functions

In [6]:
# develop any new functions here

# settings

In [7]:
job_config_path = '/g/data/es60/users/thomas_moore/code/ACDtools/job_config.yaml'
job_config_dict = ACDtools.util.load_config(job_config_path)
pprint(job_config_dict)

{'catalog_search_query_dict': {'experiment_id': 'historical',
                               'source_id': 'ACCESS-ESM1-5',
                               'table_id': 'Omon',
                               'variable_id': 'thetao'},
 'chunking': {'ACCESS_ESM15_2D': {'chunks': {'i': 360,
                                             'j': 300,
                                             'member': 1,
                                             'time': 220}},
              'ACCESS_ESM15_3D': {'chunks': {'i': -1,
                                             'j': -1,
                                             'lev': -1,
                                             'member': 1,
                                             'time': 12}}},
 'chunking_key': 'ACCESS_ESM15_3D',
 'dask_cluster': {'netcdf_work': {'memory_limit': 'None',
                                  'n_workers': 'None',
                                  'threads_per_worker': 1},
                  'zarr_work': {'memory_limit': 'N

# workflow

In [8]:
print('Running: ' + 
job_config_dict['catalog_search_query_dict']['source_id'] +
' ' + job_config_dict['catalog_search_query_dict']['experiment_id'] +
' ' + job_config_dict['catalog_search_query_dict']['variable_id'] +
' ' + job_config_dict['catalog_search_query_dict']['table_id'])

Running: ACCESS-ESM1-5 historical thetao Omon


# build workflow with ACCESS-NRI catalog that uses NCI catalog

In [11]:
catalog = intake.cat.access_nri

In [27]:
cmip6_datastore = catalog.search(name="cmip6.*", model="ACCESS-ESM1-5").to_source()

In [30]:
cmip6_datastore_searched = cmip6_datastore.search(**job_config_dict['catalog_search_query_dict'])

In [35]:
l_search = cmip6_datastore_searched.search(file_type='l')

In [36]:
f_search = cmip6_datastore_searched.search(file_type='f')

In [42]:
all_paths = cmip6_datastore_searched.unique()['path']
f_paths = f_search.unique()['path']
l_paths = l_search.unique()['path']

### build sets and compare

In [44]:
S_all, S_l, S_f = set(all_paths), set(l_paths), set(f_paths)

In [45]:
# Basic counts
print("counts:", len(S_all), len(S_l), len(S_f))

counts: 2040 1360 680


In [46]:
# Overlaps / differences
only_l   = S_l - S_f
only_f   = S_f - S_l
both_lf  = S_l & S_f
not_in_lf= S_all - (S_l | S_f)

In [47]:
print("only_l:", len(only_l), "only_f:", len(only_f), "both_lf:", len(both_lf), "in_all_not_lf:", len(not_in_lf))


only_l: 1360 only_f: 680 both_lf: 0 in_all_not_lf: 0


In [49]:
dups_all = len(all_paths) - len(S_all)
dups_l   = len(l_paths)   - len(S_l)
dups_f   = len(f_paths)   - len(S_f)
print("self-duplicates:", dups_all, dups_l, dups_f)

self-duplicates: 0 0 0


In [50]:
dup_across = (S_l & S_f) | (S_all & S_l) | (S_all & S_f)
print("duplicates across lists:", len(dup_across))

duplicates across lists: 2040


# check all catalog df df

In [53]:
DF = cmip6_datastore_searched.df

In [55]:
DF.columns

Index(['path', 'file_type', 'realm', 'frequency', 'table_id', 'project_id',
       'institution_id', 'source_id', 'experiment_id', 'member_id',
       'variable_id', 'grid_label', 'version', 'time_range'],
      dtype='object')

In [61]:
DF_f = f_search.df

In [124]:
time_ranges = DF_f['time_range'].unique().tolist()
sorted_time_ranges = sorted(time_ranges)

In [63]:
DF_f.head()

,path,file_type,realm,frequency,table_id,project_id,institution_id,source_id,experiment_id,member_id,variable_id,grid_label,version,time_range
0,/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r14i1p1f1/Omon/thetao/gn/files/d20200803/thetao_Omon_ACCESS-ESM1-5_historical_r14i1p1f1_gn_192001-192912.nc,f,ocean,mon,Omon,CMIP,CSIRO,ACCESS-ESM1-5,historical,r14i1p1f1,thetao,gn,d20200803,192001-192912
1,/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r27i1p1f1/Omon/thetao/gn/files/d20200922/thetao_Omon_ACCESS-ESM1-5_historical_r27i1p1f1_gn_192001-192912.nc,f,ocean,mon,Omon,CMIP,CSIRO,ACCESS-ESM1-5,historical,r27i1p1f1,thetao,gn,d20200922,192001-192912
2,/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r30i1p1f1/Omon/thetao/gn/files/d20200922/thetao_Omon_ACCESS-ESM1-5_historical_r30i1p1f1_gn_194001-194912.nc,f,ocean,mon,Omon,CMIP,CSIRO,ACCESS-ESM1-5,historical,r30i1p1f1,thetao,gn,d20200922,194001-194912
3,/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r8i1p1f1/Omon/thetao/gn/files/d20200529/thetao_Omon_ACCESS-ESM1-5_historical_r8i1p1f1_gn_189001-189912.nc,f,ocean,mon,Omon,CMIP,CSIRO,ACCESS-ESM1-5,historical,r8i1p1f1,thetao,gn,d20200529,189001-189912
4,/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r5i1p1f1/Omon/thetao/gn/files/d20200601/thetao_Omon_ACCESS-ESM1-5_historical_r5i1p1f1_gn_188001-188912.nc,f,ocean,mon,Omon,CMIP,CSIRO,ACCESS-ESM1-5,historical,r5i1p1f1,thetao,gn,d20200601,188001-188912


In [73]:
import pandas as pd
import numpy as np
import re

# --- 0) copy & normalize ---
df = DF.copy()

# --- 1) parse version into a datestamp (ignore 'latest') ---
# versions look like 'vYYYYMMDD' or 'dYYYYMMDD' or 'latest'
# make a datetime column 'ver_dt' (NaT for 'latest' or unparsable)
date_str = df["version"].str.extract(r"^[vd](\d{8})", expand=False)
df["ver_dt"] = pd.to_datetime(date_str, format="%Y%m%d", errors="coerce")

# tag 'latest' rows
df["is_latest_tag"] = df["version"].str.lower().eq("latest")

# --- 2) fill 'latest' rows by cross-referencing duplicated paths ---
# For each path, find the max dated version seen anywhere for that path
max_dt_for_path = (
    df.dropna(subset=["ver_dt"])
      .groupby("path")["ver_dt"].max()
)

df["ver_dt_filled"] = df["ver_dt"]
mask_latest = df["is_latest_tag"]

# fill latest rows from the path-based lookup (stays NaT if no dated peer exists)
df.loc[mask_latest, "ver_dt_filled"] = df.loc[mask_latest, "path"].map(max_dt_for_path)

# (optional) if some 'latest' rows are still NaT, you can try a fallback by (member_id,time_range):
# fallback_max_by_key = (df.dropna(subset=["ver_dt"])
#                          .groupby(["member_id","time_range"])["ver_dt"].max())
# m2 = mask_latest & df["ver_dt_filled"].isna()
# df.loc[m2, "ver_dt_filled"] = list(zip(df.loc[m2, "member_id"], df.loc[m2, "time_range"]))
# df.loc[m2, "ver_dt_filled"] = df.loc[m2, ["member_id","time_range"]].apply(
#     lambda r: fallback_max_by_key.get((r["member_id"], r["time_range"])), axis=1
# )

# --- 3) confirm that 'f' rows are the youngest per (member_id, time_range) ---
key = ["member_id", "time_range"]

# define groupwise "youngest" as max ver_dt_filled (latest date wins)
grp_max = df.groupby(key, dropna=False)["ver_dt_filled"].transform("max")

# rows that match the youngest date in their (member_id,time_range) group
df["is_group_youngest"] = df["ver_dt_filled"].eq(grp_max)

# filter to rows with file_type == 'f'
f_rows = df[df["file_type"].eq("f")].copy()

# are all 'f' rows youngest?
f_is_youngest = f_rows["is_group_youngest"].all()

print("All file_type='f' rows youngest within (member_id, time_range)?", f_is_youngest)

# if not, show the problem cases
if not f_is_youngest:
    # compare 'f' vs. group max explicitly
    probe_cols = key + ["path", "version", "ver_dt_filled"]
    not_young = f_rows[~f_rows["is_group_youngest"]][probe_cols]
    print("\nExamples where file_type='f' is NOT youngest:")
    print(not_young.head(20).to_string(index=False))

# --- 4) (optional) strict proof by joining to the group's actual latest row ---
# Build an explicit table of the youngest rows per key (there may be ties)
idx_latest_any = df.groupby(key, dropna=False)["ver_dt_filled"].transform("max").eq(df["ver_dt_filled"])
latest_any = df[idx_latest_any][key + ["path","version","file_type","ver_dt_filled"]]

# Join 'f' rows to the group's latest set and check path equality
f_vs_latest = f_rows.merge(latest_any, on=key, how="left", suffixes=("_f","_group"))
f_vs_latest["f_matches_a_group_latest"] = f_vs_latest["path_f"].eq(f_vs_latest["path_group"])

print("\n'file_type=f' paths match some group-latest path?",
      f_vs_latest["f_matches_a_group_latest"].all())

# Show mismatches if any
mismatch = f_vs_latest[~f_vs_latest["f_matches_a_group_latest"]]
if not mismatch.empty:
    print("\nMismatches (f vs group latest):")
    print(mismatch[key + ["version_f","ver_dt_filled_f","path_f","version_group","ver_dt_filled_group","path_group"]]
          .head(20).to_string(index=False))

All file_type='f' rows youngest within (member_id, time_range)? True

'file_type=f' paths match some group-latest path? False

Mismatches (f vs group latest):
member_id    time_range version_f ver_dt_filled_f                                                                                                                                                                          path_f version_group ver_dt_filled_group                                                                                                                                                                path_group
r14i1p1f1 192001-192912 d20200803      2020-08-03 /g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r14i1p1f1/Omon/thetao/gn/files/d20200803/thetao_Omon_ACCESS-ESM1-5_historical_r14i1p1f1_gn_192001-192912.nc     v20200803          2020-08-03 /g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r14i1p1f1/Omon/thetao/gn/v20200803/thetao_Omon_ACCESS-ESM1-5_historical_r14i1p1f1_g

## looking at a selected pair

In [97]:
member = "r2i1p1f1"
time_range = '188001-188912'#'198001-198912'
df_sub = cmip6_datastore_searched.df.loc[(cmip6_datastore_searched.df["member_id"] == member) & (cmip6_datastore_searched.df["time_range"]==time_range)].copy()
df_sub

,path,file_type,realm,frequency,table_id,project_id,institution_id,source_id,experiment_id,member_id,variable_id,grid_label,version,time_range
1185,/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r2i1p1f1/Omon/thetao/gn/v20191128/thetao_Omon_ACCESS-ESM1-5_historical_r2i1p1f1_gn_188001-188912.nc,l,ocean,mon,Omon,CMIP,CSIRO,ACCESS-ESM1-5,historical,r2i1p1f1,thetao,gn,v20191128,188001-188912
1480,/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r2i1p1f1/Omon/thetao/gn/latest/thetao_Omon_ACCESS-ESM1-5_historical_r2i1p1f1_gn_188001-188912.nc,l,ocean,mon,Omon,CMIP,CSIRO,ACCESS-ESM1-5,historical,r2i1p1f1,thetao,gn,latest,188001-188912
1781,/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r2i1p1f1/Omon/thetao/gn/files/d20191128/thetao_Omon_ACCESS-ESM1-5_historical_r2i1p1f1_gn_188001-188912.nc,f,ocean,mon,Omon,CMIP,CSIRO,ACCESS-ESM1-5,historical,r2i1p1f1,thetao,gn,d20191128,188001-188912


In [125]:
import pandas as pd
from itertools import product

# inputs
members = ["r1i1p1f1", "r2i1p1f1", "r14i1p1f1"]   # <- your list
members = [f"r{i}i1p1f1" for i in range(1, 41)]
time_ranges = ["188001-188912", "198001-198912"]    # <- your list
time_ranges = sorted_time_ranges

df = cmip6_datastore_searched.df

# 1) prefilter to the member/time combos you care about
df_sub = df[df["member_id"].isin(members) & df["time_range"].isin(time_ranges)].copy()

# 2) count rows per (member_id, time_range)
counts = (
    df_sub.groupby(["member_id", "time_range"])
          .size()
          .rename("n")
          .reset_index()
)

# 3) ensure we include combos that have zero rows (not present in df)
full = pd.MultiIndex.from_product(
    [members, time_ranges], names=["member_id", "time_range"]
).to_frame(index=False)

counts_full = (
    full.merge(counts, on=["member_id", "time_range"], how="left")
        .fillna({"n": 0})
        .astype({"n": int})
)

# 4) build the three lists of (member_id, time_range) tuples
exact_3   = counts_full.loc[counts_full["n"] == 3, ["member_id", "time_range"]]
less_than = counts_full.loc[counts_full["n"] < 3,  ["member_id", "time_range"]]
greater   = counts_full.loc[counts_full["n"] > 3,  ["member_id", "time_range"]]

exact_3_list   = list(map(tuple, exact_3.to_numpy()))
less_than_list = list(map(tuple, less_than.to_numpy()))
greater_list   = list(map(tuple, greater.to_numpy()))

print("== summary ==")
print("exactly 3 rows :", len(exact_3_list))
print("< 3 rows      :", len(less_than_list))
print("> 3 rows      :", len(greater_list))

# Optional: show a few examples from each bucket
print("examples (==3):", exact_3_list[:5])
print("examples (<3): ", less_than_list[:5])
print("examples (>3): ", greater_list[:5])

== summary ==
exactly 3 rows : 680
< 3 rows      : 0
> 3 rows      : 0
examples (==3): [('r1i1p1f1', '185001-185912'), ('r1i1p1f1', '186001-186912'), ('r1i1p1f1', '187001-187912'), ('r1i1p1f1', '188001-188912'), ('r1i1p1f1', '189001-189912')]
examples (<3):  []
examples (>3):  []


In [126]:
import subprocess, shutil, re
from itertools import combinations
from difflib import unified_diff
import pandas as pd

# --- safety: ensure ncdump exists
if shutil.which("ncdump") is None:
    raise RuntimeError("ncdump not found on PATH. Install netcdf-c / nco, or load the module.")

# --- your inputs from earlier cells ---
df = cmip6_datastore_searched.df
members = [f"r{i}i1p1f1" for i in range(1, 41)]
time_ranges = sorted_time_ranges  # your sorted list

# filter to the grid you care about
df_sub = df[df["member_id"].isin(members) & df["time_range"].isin(time_ranges)].copy()

# count per (member_id, time_range)
counts = (
    df_sub.groupby(["member_id","time_range"])
          .size().rename("n").reset_index()
)

# keep only pairs with exactly 3 rows
pairs_eq3 = counts.loc[counts["n"] == 3, ["member_id","time_range"]]

# --- helper: run ncdump -hs and normalize output to ignore volatile attrs ---
#    -h: header only, -s: special attributes (chunking/compression)
IGNORE_ATTR_PAT = re.compile(
    r"""
    ^\s*:\s*(
        history|
        date_created|
        creation_date|
        date_modified|
        creation_time|
        modified|
        time_written|
        tracking_id|
        uuid|
        activity_id|
        sub_experiment_id
    )\s*=
    """,
    re.IGNORECASE | re.VERBOSE,
)

# Sometimes _NCProperties or file_update_time are volatile:
IGNORE_LINE_PAT = re.compile(r"_NCProperties|file_update_time", re.IGNORECASE)

def ncdump_header(path: str) -> list[str]:
    """Return normalized ncdump -hs lines for a file."""
    out = subprocess.check_output(["ncdump", "-hs", path], text=True, stderr=subprocess.STDOUT)
    lines = out.splitlines()

    # drop lines with volatile attrs
    keep = []
    for ln in lines:
        if IGNORE_ATTR_PAT.search(ln):
            continue
        if IGNORE_LINE_PAT.search(ln):
            continue
        keep.append(ln.rstrip())

    return keep

def diff_two(a_lines: list[str], b_lines: list[str], a_label: str, b_label: str) -> str:
    """Return a small unified diff string ('' if no differences)."""
    diff = list(unified_diff(a_lines, b_lines, fromfile=a_label, tofile=b_label, n=3))
    return "\n".join(diff)

# --- iterate pairs and compare all three files pairwise ---
problems = []  # collect (member_id, time_range, pathA, pathB, difftext)

for _, row in pairs_eq3.iterrows():
    mem = row["member_id"]; tr = row["time_range"]
    rows = df_sub[(df_sub["member_id"] == mem) & (df_sub["time_range"] == tr)]
    paths = rows["path"].tolist()
    if len(paths) != 3:
        # defensive; skip if catalog changed since counts
        continue

    # run ncdump once per file
    dumps = {}
    for p in paths:
        try:
            dumps[p] = ncdump_header(p)
        except subprocess.CalledProcessError as e:
            problems.append((mem, tr, p, "", f"ncdump failed: {e}"))
            dumps[p] = None

    # pairwise compare
    for a, b in combinations(paths, 2):
        if dumps[a] is None or dumps[b] is None:
            continue
        d = diff_two(dumps[a], dumps[b], a, b)
        # if diff is non-empty after ignoring noisy attrs, record it
        if d.strip():
            problems.append((mem, tr, a, b, d))

# --- report ---
if not problems:
    print("✅ All (member_id, time_range) pairs with exactly 3 files match (no genuine header/special diffs).")
else:
    print(f"⚠️ Found {len(problems)} problematic comparisons.\nShowing first few:\n")
    for i, (mem, tr, a, b, d) in enumerate(problems[:5], 1):
        print(f"[{i}] member_id={mem}, time_range={tr}")
        print(f"    A: {a}\n    B: {b}")
        print("    --- unified diff (truncated) ---")
        # print only first ~60 lines of diff for readability
        lines = d.splitlines()
        print("\n".join(lines[:60]))
        if len(lines) > 60:
            print("    ... (diff truncated) ...")
        print()

✅ All (member_id, time_range) pairs with exactly 3 files match (no genuine header/special diffs).


# Is it as simple as `version='latest'`

In [85]:
cmip6_datastore_searched.search(version='latest')

,unique
path,680
file_type,1
realm,1
frequency,1
table_id,1
project_id,1
institution_id,1
source_id,1
experiment_id,1
member_id,40


In [ ]:
cmip6_fs38_datastore = ACDtools.util.load_cmip6_fs38_datastore()

In [ ]:
search = cmip6_fs38_datastore.search(**job_config_dict['catalog_search_query_dict'])

In [ ]:
search

In [ ]:
%%time
print('Running: ' + job_config_dict['catalog_search_query_dict']['source_id'] + ' ' + job_config_dict['catalog_search_query_dict']['experiment_id'] + ' ' + job_config_dict['catalog_search_query_dict']['variable_id'] + ' ' + job_config_dict['catalog_search_query_dict']['table_id'])
# load catalog
cmip6_fs38_datastore = load_cmip6_CLEX_datastore()
# search catalog for list of files
search = cmip6_fs38_datastore.search(**job_config_dict['catalog_search_query_dict'])
# load into one object using xarray kwags for chunking and handling cftime
####
#ds = load_ACCESS_ESM(search,use_cftime=True,chunking_key=job_config_dict['chunking_key'])
####
ds = load_ACCESS_ESM_ensemble(search,use_cftime=True,chunking_key=job_config_dict['chunking_key'])
# save and drop multidimensional coordinates
ds = save_n_drop_multidim_lat_lon(ds,save_coords_dir=job_config_dict['paths']['save_coords_dir'],variable_name = job_config_dict['catalog_search_query_dict']['variable_id'])
# remove encoding
remove_encoding(ds)
# write out zarr
current_datetime = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
print('Started zarr write at '+current_datetime)
filename = f"{job_config_dict['paths']['write_dir']}\
{job_config_dict['catalog_search_query_dict']['source_id']}\
.{job_config_dict['catalog_search_query_dict']['experiment_id']}\
.{job_config_dict['catalog_search_query_dict']['variable_id']}\
.v{current_datetime}.zarr"
ds.to_zarr(filename,consolidated=True)
print('Finished at '+ current_datetime)

In [ ]:
import xarray as xr

# setup LocalCluster()

In [ ]:
from dask.distributed import Client, LocalCluster, progress
cluster = LocalCluster()
client = Client(cluster)

# Load NCI catalog
Follow documentation here: https://opus.nci.org.au/pages/viewpage.action?pageId=213713098

In [ ]:
import intake
cmip6_fs38_cat = intake.open_esm_datastore("/g/data/dk92/catalog/v2/esm/cmip6-fs38/catalog.json")

# BELOW cell is manual process of editing the member_id's

# load 10 members of monthly `o2` into one dataset `dict` and concatinate over ensemble members

In [ ]:
cmip6_fs38_cat_filtered = cmip6_fs38_cat.search(
    source_id="ACCESS-ESM1-5", 
    experiment_id="historical", 
    table_id="Omon", 
    file_type="f",
    variable_id = "thetao"
)

In [ ]:
cmip6_fs38_cat_filtered

#### what is chunking on NetCDF files? `'chunksizes': (1, 25, 150, 180)`

# DS_dict

In [ ]:
xarray_open_kwargs = {"chunks": {"time": 12, "lev": 50, "i" : 360,'j' :300}}

In [ ]:
%%time
DS_dict = cmip6_fs38_cat_filtered.to_dataset_dict(progressbar=False,xarray_open_kwargs=xarray_open_kwargs)

In [ ]:
%%time
DS_concat = xr.concat(
    DS_dict.values(), 
    dim="member"
)

# can we simplify to a single DataArray?

In [ ]:
DA_concat = DS_concat['thetao']
DA_concat

In [ ]:
DS_out = DA_concat.to_dataset()
DS_out

# drop multi-dimensional coordinates

In [ ]:
DS_out = DS_out.drop(['latitude','longitude'])

In [ ]:
DS_out

# remove encoding

In [ ]:
def remove_zarr_encoding(DS):
    for var in DS:
        DS[var].encoding = {}

    for coord in DS.coords:
        DS[coord].encoding = {}
    return DS

In [ ]:
DS_out.thetao.encoding

In [ ]:
remove_zarr_encoding(DS_out)

In [ ]:
DS_out.thetao.encoding

In [ ]:
file_path = '/scratch/es60/ard/models/ACCESS-ESM15/ARD/'

In [ ]:
%%time
DS_out.to_zarr(file_path+'ACCESS-ESM1.5-historical-thetao.zarr',consolidated=True)

In [ ]:
result = xr.open_zarr(file_path+'ACCESS-ESM1.5-historical-thetao.zarr',consolidated=True)

In [ ]:
result

In [ ]:
temp = result.thetao

In [ ]:
%%time
temp.isel({'member':0,'lev':5}).isel(time=slice(0,12)).mean('time').plot(robust=True)

#### The End (?)

# can we rechunk 40 member collections to `for member`?

In [ ]:
loaded_member1_40_rc = loaded_member1_40.chunk({'member':-1,'time':1,'lev':25})
loaded_member1_40_rc

In [ ]:
remove_zarr_encoding(loaded_member1_40_rc)

In [ ]:
loaded_member1_40_rc.o2.encoding

In [ ]:
%%time
loaded_member1_40_rc.to_zarr(file_path+'r1-40.ACCESS-ESM1.5-historical-o2-chunk4member.zarr',consolidated=True)

In [ ]:
chunk4member = xr.open_zarr(file_path+'r1-40.ACCESS-ESM1.5-historical-o2-chunk4member.zarr',consolidated=True)

In [ ]:
chunk4member

In [ ]:
chunk4member.o2.max('member').isel(lev=0).mean(dim=['i','j']).plot()

# can we chunk for time?

In [ ]:
loaded_member1_40_rc = loaded_member1_40.chunk({'time':-1,'lev':1,'i':150})
loaded_member1_40_rc

In [ ]:
remove_zarr_encoding(loaded_member1_40_rc)

In [ ]:
loaded_member1_40_rc.o2.encoding

In [ ]:
%%time
loaded_member1_40_rc.to_zarr(file_path+'r1-40.ACCESS-ESM1.5-historical-o2-chunk4time.zarr',consolidated=True)

In [ ]:
chunked_for_time = xr.open_zarr(file_path+'r1-40.ACCESS-ESM1.5-historical-o2-chunk4time.zarr',consolidated=True)

In [ ]:
chunked_for_time

In [ ]:
chunked_for_time.o2.mean('time').isel(lev=0).isel(member=0).plot()

# can we chunk for `time & member`?

In [ ]:
chunked_for_time_rc = chunked_for_time.chunk({'time':-1,'lev':1,'member':-1,'i':36,'j':30})
chunked_for_time_rc

In [ ]:
remove_zarr_encoding(chunked_for_time_rc)

In [ ]:
chunked_for_time_rc.o2.encoding

In [ ]:
%%time
chunked_for_time_rc.to_zarr(file_path+'r1-40.ACCESS-ESM1.5-historical-o2-chunk4time4member.zarr',consolidated=True)

In [ ]:
chunked_for_time_member = xr.open_zarr(file_path+'r1-40.ACCESS-ESM1.5-historical-o2-chunk4time4member.zarr',consolidated=True)

In [ ]:
chunked_for_time_member

# chunk for 3D space

In [ ]:
loaded_member1_40_rc = loaded_member1_40.chunk({'i':-1,'j':-1,'lev':50,'time':24,'member':1})
loaded_member1_40_rc

In [ ]:
remove_zarr_encoding(loaded_member1_40_rc)

In [ ]:
%%time
loaded_member1_40_rc.to_zarr(file_path+'r1-40.ACCESS-ESM1.5-historical-o2-chunk4xyz.zarr',consolidated=True)

In [ ]:
chuned_xyz = xr.open_zarr(file_path+'r1-40.ACCESS-ESM1.5-historical-o2-chunk4xyz.zarr',consolidated=True)

In [ ]:
chuned_xyz

In [ ]:
chuned_xyz.o2.mean(dim=['i','j','lev']).plot.line(x="time")

# can we chunk for lev & time?

In [ ]:
chunked_for_time_rc = chunked_for_time.chunk({'time':-1,'lev':-1,'member':1,'i':36,'j':30})
chunked_for_time_rc

In [ ]:
remove_zarr_encoding(chunked_for_time_rc)

In [ ]:
%%time
chunked_for_time_rc.to_zarr(file_path+'r1-40.ACCESS-ESM1.5-historical-o2-chunk4time4z.zarr',consolidated=True)

In [ ]:
chunked_time_z = xr.open_zarr(file_path+'r1-40.ACCESS-ESM1.5-historical-o2-chunktime4z.zarr',consolidated=True)

In [ ]:
! rm -rf /scratch/xv83/tm4888/tmp_rechunker/*.zarr # clean up any existing temporary data

In [ ]:
from rechunker import rechunk
target_chunks = {
    "o2": {'member':3,'time':12,'i':360,'j':300,'lev':25},
    "time": None,  # don't rechunk this array
    "i":None,
    "j":None,
    "lev":None,
    "latitude":None,
    "longitude":None
}
max_mem = "2000MB"

target_store = "/scratch/xv83/tm4888/tmp_rechunker/group_rechunked.zarr"
temp_store = "/scratch/xv83/tm4888/tmp_rechunker/group_rechunked-tmp.zarr"
source_group = DS_out

group_plan = rechunk(
    source_group, target_chunks, max_mem, target_store, temp_store=temp_store)
group_plan

In [ ]:
%%time
group_plan.execute()

# Can we open and use this zarr collection?

In [ ]:
data = xr.open_zarr('/scratch/xv83/tm4888/tmp_rechunker/group_rechunked.zarr',consolidated=True)

# can we write to large `zarr` collection?

In [ ]:
write_path = '/scratch/xv83/tm4888/zarr_collections/ACCESS_ESM1.5/'

In [ ]:
DS_out = DA_concat_rc.to_dataset()
DS_out

# remove encoding

In [ ]:
for var in DS_out:
    DS_out.encoding = {}

for coord in DS_out.coords:
    DS_out.encoding = {}

In [ ]:
%%time
DS_out.to_zarr(write_path + 'DS_out.zarr',consolidated=True)

# load back in 

In [ ]:
reloaded_data = xr.open_zarr(write_path + 'DS_out.zarr',consolidated=True)

In [ ]:
o2_historical_data

# remove encoding

In [ ]:
for var in o2_historical_data:
    o2_historical_data[var].encoding = {}

for coord in o2_historical_data.coords:
    o2_historical_data[coord].encoding = {}

In [ ]:
o2_historical_surface = o2_historical_data.isel(lev=0)

In [ ]:
o2_historical_surface

In [ ]:
o2_historical_surface_rc = o2_historical_surface.chunk({'member':-1,'time':-1,'i':36,'j':30})

In [ ]:
o2_historical_surface_rc

In [ ]:
%%time
o2_historical_surface_rc.to_zarr(write_path + 'ACCESS_ESM1.5_o2_40_member_surface_chunk4member4time.zarr',consolidated=True)

In [ ]:
data = xr.open_zarr(write_path + 'ACCESS_ESM1.5_o2_40_member_surface_chunk4member4time.zarr',consolidated=True)

In [ ]:
data

In [ ]:
result = (data.o2.max(dim = ['member','time']) - data.o2.mean(dim = ['member','time'])).compute()

In [ ]:
result.plot(robust=True)

In [ ]:
o2_emean = o2_historical_data.mean('member')

In [ ]:
o2_emean

In [ ]:
%%time
o2_emean.to_zarr(write_path + 'ACCESS_ESM1.5_o2_member_mean.zarr',consolidated=True)

# rechunk over member for some box

In [ ]:
box_data = DS_o2_monthly_hist.o2.isel({"lev":10}).sel(i=slice(60,150)).sel(j=slice(60,150))

In [ ]:
box_data

In [ ]:
box_data_rc = box_data.chunk({'member':-1,'time':200})

In [ ]:
box_data_rc

In [ ]:
%%time
box_data_rc.to_netcdf('/scratch/xv83/tm4888/PAC-TUNA/box_data_rc.nc')

In [ ]:
%%time
e_mean = box_data.mean(dim="member").compute()

# ----------------------------

In [ ]:
box_data_rc = xr.open_mfdataset('/scratch/xv83/tm4888/PAC-TUNA/box_data_rc.nc',parallel=True)

In [ ]:
box_data_rc

In [ ]:
%%time
emean = box_data_rc.mean('member').compute()

# Load the production ACCESS-NRI catalogue

In [ ]:
import intake

cat = intake.cat.access_nri

esm_ds = cat["cmip6_fs38"]

esm_ds.esmcat.aggregation_control.groupby_attrs = [
    attrs for attrs in esm_ds.esmcat.aggregation_control.groupby_attrs
    if attrs != "version"
]

esm_ds_filtered = esm_ds.search(
    source_id="ACCESS-ESM1-5", 
    experiment_id="ssp370", 
    table_id="Omon", 
    variable_id="o2", 
    file_type="l"
)

In [ ]:
esm_ds_filtered.unique()['version']

In [ ]:
esm_ds_filtered

In [ ]:
DS_dict = esm_ds_filtered.to_dataset_dict(progressbar=False)
DS_dict

In [ ]:
DS_dict

In [ ]:
DS_o2_monthly_ssp370 = xr.concat(
    DS_dict.values(), 
    dim="member"
)

In [ ]:
DS_o2_monthly_ssp370

In [ ]:
result = DS_o2_monthly_ssp370.o2.isel(lev=20).mean(dim=['i','j'])

In [ ]:
result

In [ ]:
result.plot()

In [ ]:
ds

In [ ]:
esm_ds_filtered['l.CMIP.CSIRO.ACCESS-ESM1-5.ssp585.r9i1p1f1.mon.ocnBgchem.Omon.o2.gn'].df

In [ ]:
esm_ds_filtered.keys()

In [ ]:
esm_ds_filtered['l.CMIP.CSIRO.ACCESS-ESM1-5.ssp585.r40i1p1f1.mon.ocnBgchem.Omon.o2.gn'].df

In [ ]:
import intake

nri_cat = intake.cat.access_nri

In [ ]:
cmip6_fs38_cat = nri_cat['cmip6_fs38']
cmip6_fs38_cat

In [ ]:
ScenarioMIP6_fs38_cat = cmip6_fs38_cat.search(path='/g/data/fs38/publications/CMIP6/ScenarioMIP/*')

In [ ]:
ScenarioMIP6_fs38_cat.unique()['realm']

In [ ]:
ScenarioMIP6_fs38_oceanBGC_cat = ScenarioMIP6_fs38_cat.search(realm=['ocnBgchem'])

In [ ]:
ScenarioMIP6_fs38_oceanBGC_cat

In [ ]:
ScenarioMIP6_fs38_o2_cat = ScenarioMIP6_fs38_oceanBGC_cat.search(variable_id='o2')
ScenarioMIP6_fs38_o2_cat

In [ ]:
ScenarioMIP6_fs38_o2_cat.unique()['frequency']

In [ ]:
ScenarioMIP6_fs38_monthly_o2_cat = ScenarioMIP6_fs38_o2_cat.search(frequency='mon')
ScenarioMIP6_fs38_monthly_o2_cat

In [ ]:
ScenarioMIP6_fs38_monthly_o2_cat.unique()['experiment_id']

In [ ]:
ScenarioMIP6_fs38_ssp585_monthly_o2_cat = ScenarioMIP6_fs38_monthly_o2_cat.search(experiment_id = 'ssp585')
ScenarioMIP6_fs38_ssp585_monthly_o2_cat

In [ ]:
ScenarioMIP6_fs38_ssp585_monthly_o2_cat.unique()['version']

In [ ]:
ScenarioMIP6_fs38_ssp585_monthly_o2_40e_cat = ScenarioMIP6_fs38_ssp585_monthly_o2_cat.search(version=['v20210831','v20210318'])
ScenarioMIP6_fs38_ssp585_monthly_o2_40e_cat

In [ ]:
ScenarioMIP6_fs38_ssp585_monthly_o2_d20210831_cat = ScenarioMIP6_fs38_ssp585_monthly_o2_cat.search(version='d20210831')
ScenarioMIP6_fs38_ssp585_monthly_o2_d20210831_cat

In [ ]:
ScenarioMIP6_fs38_ssp585_monthly_o2_v20210831_cat.df.head()

In [ ]:
ScenarioMIP6_fs38_ssp585_monthly_o2_v20210831_cat.unique()['time_range']

In [ ]:
ScenarioMIP6_fs38_ssp585_monthly_o2_v20210831_cat.unique()['member_id']

In [ ]:
ScenarioMIP6_fs38_ssp585_monthly_o2_v20210831_r40i1p1f1_cat = ScenarioMIP6_fs38_ssp585_monthly_o2_v20210831_cat.search(member_id = 'r40i1p1f1')
ScenarioMIP6_fs38_ssp585_monthly_o2_v20210831_r40i1p1f1_cat

In [ ]:
dataset = ScenarioMIP6_fs38_ssp585_monthly_o2_v20210831_r40i1p1f1_cat.to_dask(progressbar=False)

In [ ]:
dataset

In [ ]:
dataset['o2'].encoding

In [ ]:
o2_chunked_for_time = dataset.o2.chunk({'time':-1,'lev':25,'i':50,'j':18})

In [ ]:
o2_chunked_for_time

In [ ]:
o2_200m = o2_chunked_for_time.isel(lev=slice(0,21))

In [ ]:
o2_200m

In [ ]:
write_path = '/scratch/xv83/tm4888/zarr_collections/'

In [ ]:
o2_200m.to_zarr(write_path + 'ACCESS_ESM1.5_o2_200m_r40i1p1f1.zarr',consolidated=True)

# read back in

In [ ]:
o2 = xr.open_zarr(write_path + 'ACCESS_ESM1.5_o2_200m_r40i1p1f1.zarr',consolidated=True)

In [ ]:
o2

In [ ]:
o2_clim = o2.groupby('time.month').mean()

In [ ]:
o2_clim

In [ ]:
%%time
o2_clim = o2_clim.compute()

In [ ]:
o2_anom = o2.groupby('time.month') - o2_clim

In [ ]:
o2_anom

In [ ]:
PLOT_DATA = o2_anom.o2.isel(time=1031,lev=20)

In [ ]:
%%time
PLOT_DATA = PLOT_DATA.compute()

In [ ]:
import cartopy.crs as ccrs
plt.figure(figsize=(14, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
PLOT_DATA.plot.pcolormesh(
    ax=ax, transform=ccrs.PlateCarree(), x="longitude", y="latitude"
)
ax.coastlines()
plt.title('o2 December anomaly year 2100\n 205m \n ACCESS_ESM1.5_o2_200m_r40i1p1f1')

In [ ]:
diff = dataset.o2.isel(time=0) - dataset.o2.isel(time=1031)

In [ ]:
diff

In [ ]:
yearly_mean = dataset

In [ ]:
mean_depth_diff = diff.mean('lev')
mean_depth_diff

In [ ]:
import cartopy.crs as ccrs
plt.figure(figsize=(14, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
mean_depth_diff.plot.pcolormesh(
    ax=ax, transform=ccrs.PlateCarree(), x="longitude", y="latitude"
)
ax.coastlines()
plt.title('depth mean O2 // 

In [ ]:
mean_depth_diff.plot(x=mean_depth_diff['longitude'],y=mean_depth_diff['latitude'])

In [ ]:
source_dict = ScenarioMIP6_fs38_ssp585_monthly_o2_v20210831_cat.to_source_dict()

In [ ]:
dataset_dict

In [ ]:
dataset_dict.keys()

In [ ]:
ESM15_o2_cat = cmip6_fs38_cat.search(path='/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/*',variable_id = 'o2')

In [ ]:
ESM15_o2_cat

In [ ]:
ESM15_o2_cat.unique()['experiment_id']

In [ ]:
ESM15_o2_cat.unique()['frequency']

In [ ]:
ESM15_monthly_o2_cat = ESM15_o2_cat.search(frequency='mon')

In [ ]:
ESM15_monthly_o2_cat

In [ ]:
ESM15_monthly_o2_cat.unique()['experiment_id']

# Extension BGC run `01deg_jra55v140_iaf_cycle4_jra55v150_extension`

In [ ]:
BGC_extension_cat = nri_cat['01deg_jra55v140_iaf_cycle4_jra55v150_extension']
BGC_extension_cat

# Can we build monthly (or daily) datasets for TEMP and O2 ?

In [ ]:
# show nri cat metadata
nri_cat.metadata

In [ ]:
BGC_extension_cat.search(variable='o2').keys()

In [ ]:
BGC_extension_cat.search(file_id='oceanbgc_3d_o2_1_monthly_mean_4_sigfig_ym_XXXX_XX')

In [ ]:
BGC_extension_cat.df.head()

In [ ]:
monthly_o2 = BGC_extension_cat.search(file_id='oceanbgc_3d_o2_1_monthly_mean_4_sigfig_ym_XXXX_XX')

In [ ]:
monthly_o2_data = monthly_o2.to_dask(xarray_open_kwargs=dict(chunks={'time':100,'yt_ocean':540,'xt_ocean':720}))

In [ ]:
monthly_o2_data

In [ ]:
monthly_o2_data.o2.isel({'time':0,'st_ocean':10}).plot()

In [ ]:
daily_o2_extension_search = BGC_extension_cat.search(realm='ocean',frequency='1day',variable=['o2'])

In [ ]:
daily_o2_extension_search

In [ ]:
daily_extension_search = BGC_extension_cat.search(realm='ocean',frequency='1day')

In [ ]:
daily_extension_search

In [ ]:
daily_extension_search.unique()['variable']

In [ ]:
temp_o2_extension_data = temp_o2_extension_search.to_dask(xarray_open_kwargs=dict(chunks={'time':100,'yt_ocean':540,'xt_ocean':720}))

In [ ]:
temp_o2_extension_data

### MLD test run - COSIMA

In [ ]:
mld_extension_search = BGC_extension_cat.search(realm='ocean',frequency='1day',variable='mld')
mld_extension_search

In [ ]:
mld_extension_data = mld_extension_search.to_dask(xarray_open_kwargs=dict(chunks={'time':100,'yt_ocean':540,'xt_ocean':720}))

In [ ]:
mld_extension_data

# Base BGC run `01deg_jra55v140_iaf_cycle4`

In [ ]:
BGC_cat = cat['01deg_jra55v140_iaf_cycle4']
BGC_cat

In [ ]:
mld_search = BGC_cat.search(realm='ocean',frequency='1day',variable='mld')
mld_search

In [ ]:
mld_search.df

In [ ]:
mld_data = mld_search.to_dask(xarray_open_kwargs=dict(chunks={'time':100,'yt_ocean':540,'xt_ocean':720}))

In [ ]:
mld_data

# What do these individual DS's look like?

In [ ]:
mld_data

In [ ]:
mld_extension_data

# Concat base and extension MLD data

In [ ]:
%%time
mld_DS = xr.concat([mld_data,mld_extension_data],dim='time')

In [ ]:
mld_DS

# can we write this to `zarr` on scratch?

In [ ]:
rechunked_mld_DS = mld_DS.chunk({'time':100})

In [ ]:
rechunked_mld_DS

In [ ]:
write_path = '/scratch/xv83/tm4888/zarr_collections/'

In [ ]:
%%time
rechunked_mld_DS.to_zarr(write_path+'daily.mld.concat.chunk.100-540-720.zarr', consolidated = True)

In [ ]:
test_read = xr.open_zarr(write_path+'daily.mld.concat.chunk.100-540-720.zarr', consolidated = True)

In [ ]:
test_read

In [ ]:
point = test_read.mld.max(dim=['xt_ocean','yt_ocean'])

In [ ]:
point

In [ ]:
point.plot()

In [ ]:
client.close()